# Pre generated model: Zincblende

Here we run "step 4" separately to define a common object for all materials of the same type.

This is useful because each time you run QSYMM to define the kp model, its numerical process may lead Hamiltonian families that are equivalent, but with different numerical prefactors or different order of the $c_n$ coefficients. Therefore, here define the model and store it in a file to be read by the examples that uses this crystal symmetry.

In [79]:
import numpy as np
import pydft2kp as dft2kp
from qsymm.hamiltonian_generator import hamiltonian_from_family
import sympy
import pickle

In [80]:
# step 4: builds optimal model with qsymm
# Pauli matrices
from pydft2kp.constants import s0, sx, sy, sz

# build symmetry operations of Td = {C3, S4}
# consider also T = time-reversal symmetry

# ------------------------------------
U6 = 0.5*np.array([[1-1j, -1-1j],[1-1j, 1+1j]])
U7 = 0.5*np.array([[1+1j, 1-1j],[-1-1j, 1-1j]])
U8 = np.array([
    [-1-1j, 1-1j, (1+1j)*np.sqrt(3), (-1+1j)*np.sqrt(3)],
    [-1-1j, -1+1j, (1+1j)*np.sqrt(3), (1-1j)*np.sqrt(3)],
    [(-1-1j)*np.sqrt(3), (1-1j)*np.sqrt(3), -1-1j, 1-1j],
    [(-1-1j)*np.sqrt(3), (-1+1j)*np.sqrt(3), -1-1j, -1+1j]
    ])/4
U = np.block([[U6, np.zeros([2,4]), np.zeros([2,2])],
              [np.zeros([4,2]), U8, np.zeros([4,2])],
              [np.zeros([2,2]), np.zeros([2,4]), U7]])
C3 = dft2kp.rotation(1/3, [1,1,1], U=U)
# ------------------------------------

# ------------------------------------
U6 = np.diag([1-1j, 1+1j])/np.sqrt(2)
U7 = np.diag([-1-1j, -1+1j])/np.sqrt(2)
U8 = np.diag([1+1j, 1-1j, -1-1j, -1+1j])/np.sqrt(2)
U = np.block([[U6, np.zeros([2,4]), np.zeros([2,2])],
              [np.zeros([4,2]), U8, np.zeros([4,2])],
              [np.zeros([2,2]), np.zeros([2,4]), U7]])
S4 = dft2kp.rotation(1/4, [0,0,1], U=U, inversion=True)
# ------------------------------------

# ------------------------------------
U6 = 1j*sy
U7 = 1j*sy
U8 = np.kron(s0, -1j*sy)
U = np.block([[U6, np.zeros([2,4]), np.zeros([2,2])],
              [np.zeros([4,2]), U8, np.zeros([4,2])],
              [np.zeros([2,2]), np.zeros([2,4]), U7]])
TRS = dft2kp.time_reversal(3, U=U)
# ------------------------------------

symms = [C3, S4, TRS]
qs = dft2kp.qsymm(symms, dim=3, total_power=2);

Matrix([
[                                              c0 + c7*k_x**2 + c7*k_y**2 + c7*k_z**2,                                                                                     0,                                            I*c15*k_x*k_z + c15*k_y*k_z - sqrt(3)*I*c5*k_x/2 + sqrt(3)*c5*k_y/2,                                                                 -I*c12*k_x**2/2 - I*c12*k_y**2/2 + I*c12*k_z**2,                                         sqrt(3)*I*c15*k_x*k_z/3 - sqrt(3)*c15*k_y*k_z/3 + I*c5*k_x/2 + c5*k_y/2,                          -sqrt(3)*I*c12*k_x**2/2 + sqrt(3)*I*c12*k_y**2/2 - 2*sqrt(3)*c15*k_x*k_y/3 + I*c5*k_z,                                      I*c16*k_x*k_z - c16*k_y*k_z + I*c6*k_x + c6*k_y,                                                              -c16*k_x*k_y + I*c6*k_z],
[                                                                                   0,                                                c0 + c7*k_x**2 + c7*k_y**2 + c7*k_z**2,                     

In [81]:
# save symms and qs object
with open('zincblende.pickle', 'wb') as fp:
  pickle.dump(symms, fp)
  pickle.dump(qs, fp)

# Organazing H to print as latex for paper

In [3]:
# import and read existing data
# to avoid running model above again if needed

from qsymm.hamiltonian_generator import hamiltonian_from_family
import sympy
import pickle

import pickle
# load symms and qs object
with open('zincblende.pickle', 'rb') as fp:
  symms = pickle.load(fp)
  qs = pickle.load(fp)

In [47]:
# print model in Latex format
H = hamiltonian_from_family(qs.model, nsimplify=True)

cs = sympy.symbols('c:17')
kx, ky, kz = sympy.symbols('k_x, k_y, k_z')
k2, kp, km = sympy.symbols('k^2, k_+, k_-')
kxy2 = sympy.symbols('k_{\parallel}^2') # = kx**2 + ky**2
Kbig = sympy.symbols('\hat{K}') # = kx**2 - ky**2
# kp = kx+i.ky
# km = kx-i.ky
# kx = (kp+km)/2
# ky = (kp-km)/2i

H = H.applyfunc(lambda ij: sympy.collect(ij, cs))

H = sympy.simplify(H)
H = sympy.simplify(H.subs({kx**2 + ky**2 + kz**2: k2}))
H = sympy.simplify(H.subs({kx**2 + ky**2: kxy2}))
H = sympy.simplify(H.subs({kx+sympy.I*ky: kp, 
                           kx-sympy.I*ky: km,
                           sympy.I*kx-ky: sympy.I*kp, 
                           sympy.I*kx+ky: sympy.I*km,
                           }))

H = sympy.simplify(H.subs({kx**2 - ky**2: Kbig}))

H

Matrix([
[                                                 c0 + c7*k^2,                                                             0,                                   I*(c15*k_-*k_z - sqrt(3)*c5*k_+/2),                                 I*c12*(2*k_z**2 - k_{\parallel}^2)/2,                                 I*(sqrt(3)*c15*k_+*k_z/3 + c5*k_-/2),        -sqrt(3)*I*\hat{K}*c12/2 - 2*sqrt(3)*c15*k_x*k_y/3 + I*c5*k_z,                         I*(c16*k_+*k_z + c6*k_-),                           -c16*k_x*k_y + I*c6*k_z],
[                                                           0,                                                   c0 + c7*k^2,                                I*c12*(-2*k_z**2 + k_{\parallel}^2)/2,                                   I*(c15*k_+*k_z + sqrt(3)*c5*k_-/2),         sqrt(3)*I*\hat{K}*c12/2 - 2*sqrt(3)*c15*k_x*k_y/3 + I*c5*k_z,                                 I*(sqrt(3)*c15*k_-*k_z/3 - c5*k_+/2),                           c16*k_x*k_y - I*c6*k_z,                         I*(

In [51]:
print(sympy.latex(H))

\left[\begin{matrix}c_{0} + c_{7} k^{2} & 0 & i \left(c_{15} k_{-} k_{z} - \frac{\sqrt{3} c_{5} k_{+}}{2}\right) & \frac{i c_{12} \cdot \left(2 k_{z}^{2} - k_{\parallel}^2\right)}{2} & i \left(\frac{\sqrt{3} c_{15} k_{+} k_{z}}{3} + \frac{c_{5} k_{-}}{2}\right) & - \frac{\sqrt{3} i \hat{K} c_{12}}{2} - \frac{2 \sqrt{3} c_{15} k_{x} k_{y}}{3} + i c_{5} k_{z} & i \left(c_{16} k_{+} k_{z} + c_{6} k_{-}\right) & - c_{16} k_{x} k_{y} + i c_{6} k_{z}\\0 & c_{0} + c_{7} k^{2} & \frac{i c_{12} \left(- 2 k_{z}^{2} + k_{\parallel}^2\right)}{2} & i \left(c_{15} k_{+} k_{z} + \frac{\sqrt{3} c_{5} k_{-}}{2}\right) & \frac{\sqrt{3} i \hat{K} c_{12}}{2} - \frac{2 \sqrt{3} c_{15} k_{x} k_{y}}{3} + i c_{5} k_{z} & i \left(\frac{\sqrt{3} c_{15} k_{-} k_{z}}{3} - \frac{c_{5} k_{+}}{2}\right) & c_{16} k_{x} k_{y} - i c_{6} k_{z} & i \left(- c_{16} k_{-} k_{z} + c_{6} k_{+}\right)\\i \left(- c_{15} k_{+} k_{z} + \frac{\sqrt{3} c_{5} k_{-}}{2}\right) & \frac{i c_{12} \cdot \left(2 k_{z}^{2} - k_{\parallel}^